# 🔥 TacticalFitAI - Real Data Collection
## ดึงข้อมูลนักเตะจริงด้วย soccerdata

**วัตถุประสงค์:** ดึงข้อมูล Forwards จากลีกใหญ่ 5 ลีก (Premier League, La Liga, Serie A, Bundesliga, Ligue 1)

**ข้อมูลที่ต้องการ:**
- Player name
- Position
- Finishing (Shots, Goals)
- Positioning (xG, On-target shots)
- Speed (Sprint speed - อาจต้องใช้ SoFIFA)
- Strength (Aerial duels won)
- Passing (Pass completion, Key passes)
- xG (Expected Goals)
- PressActions (Pressures)
- Vision (Progressive passes)
- Aggression (Fouls, Yellow cards)
- Composure (Penalty conversion)
- OffTheBall (Progressive distance)

## 📦 Step 1: ติดตั้ง Libraries

In [ ]:
# ติดตั้ง soccerdata
!pip install soccerdata -q
!pip install pandas numpy -q

print("✅ Installation complete!")

## 🌍 Step 2: Import Libraries และ Setup

In [ ]:
import soccerdata as sd
import pandas as pd
import numpy as np
from datetime import datetime

print(f"📊 Soccerdata version: {sd.__version__}")
print(f"🐼 Pandas version: {pd.__version__}")

## 🏆 Step 3: ดึงข้อมูลจาก FBref (สถิติจริง)

In [ ]:
# กำหนดลีกและฤดูกาล
leagues = [
    "ENG-Premier League",
    "ESP-La Liga",
    "ITA-Serie A",
    "GER-Bundesliga",
    "FRA-Ligue 1"
]

# ใช้ฤดูกาลล่าสุด (2024-2025)
season = "2024-2025"

print(f"🔄 Fetching data from {len(leagues)} leagues for {season} season...")
print("⏳ This may take 2-3 minutes...\n")

### 3.1 ดึงข้อมูล Standard Stats (Goals, Assists, Minutes)

In [ ]:
# สร้าง FBref scraper
fbref = sd.FBref(leagues=leagues, seasons=season)

# ดึง standard stats
print("📥 Fetching standard stats...")
try:
    standard_stats = fbref.read_player_season_stats(stat_type="standard")
    print(f"✅ Got {len(standard_stats)} player records")
    print(f"📋 Columns: {list(standard_stats.columns[:10])}...\n")
except Exception as e:
    print(f"❌ Error: {e}")
    print("⚠️ หาก error ให้ลองใช้ season='2023-2024' แทน")

### 3.2 ดึงข้อมูล Shooting Stats (Shots, xG, Goals)

In [ ]:
print("📥 Fetching shooting stats...")
try:
    shooting_stats = fbref.read_player_season_stats(stat_type="shooting")
    print(f"✅ Got {len(shooting_stats)} player records")
except Exception as e:
    print(f"❌ Error: {e}")

### 3.3 ดึงข้อมูล Passing Stats

In [ ]:
print("📥 Fetching passing stats...")
try:
    passing_stats = fbref.read_player_season_stats(stat_type="passing")
    print(f"✅ Got {len(passing_stats)} player records")
except Exception as e:
    print(f"❌ Error: {e}")

### 3.4 ดึงข้อมูล Defensive Stats (Pressures, Tackles)

In [ ]:
print("📥 Fetching defensive stats...")
try:
    defensive_stats = fbref.read_player_season_stats(stat_type="defense")
    print(f"✅ Got {len(defensive_stats)} player records")
except Exception as e:
    print(f"❌ Error: {e}")

### 3.5 ดึงข้อมูล Possession Stats (Progressive carries, etc.)

In [ ]:
print("📥 Fetching possession stats...")
try:
    possession_stats = fbref.read_player_season_stats(stat_type="possession")
    print(f"✅ Got {len(possession_stats)} player records")
except Exception as e:
    print(f"❌ Error: {e}")

## 🔗 Step 4: Merge ข้อมูลทั้งหมด

In [ ]:
# Merge datasets
print("🔗 Merging all datasets...")

# Start with standard stats
merged_df = standard_stats.copy()

# Merge shooting
if 'shooting_stats' in locals():
    merged_df = merged_df.merge(shooting_stats, left_index=True, right_index=True, how='left', suffixes=('', '_shoot'))

# Merge passing
if 'passing_stats' in locals():
    merged_df = merged_df.merge(passing_stats, left_index=True, right_index=True, how='left', suffixes=('', '_pass'))

# Merge defensive
if 'defensive_stats' in locals():
    merged_df = merged_df.merge(defensive_stats, left_index=True, right_index=True, how='left', suffixes=('', '_def'))

# Merge possession
if 'possession_stats' in locals():
    merged_df = merged_df.merge(possession_stats, left_index=True, right_index=True, how='left', suffixes=('', '_poss'))

print(f"✅ Merged dataset shape: {merged_df.shape}")
print(f"📊 Total columns: {len(merged_df.columns)}")

## ⚽ Step 5: Filter เฉพาะ Forwards

In [ ]:
# Reset index เพื่อให้เข้าถึง columns ได้
df = merged_df.reset_index()

# ดูว่า column ชื่ออะไรบ้าง
print("📋 Available columns:")
print(df.columns.tolist()[:20], "...\n")

# FBref uses MultiIndex columns (tuples)
# Need to flatten or access by tuple
print("🔍 Checking column structure...")
if isinstance(df.columns[0], tuple):
    print("✅ MultiIndex detected - flattening columns...")
    # Flatten column names
    df.columns = ['_'.join(col).strip('_') if isinstance(col, tuple) else col for col in df.columns]
    print(f"📋 Flattened columns: {df.columns.tolist()[:10]}...\n")

# Filter forwards (FW, ST, CF)
forward_keywords = ['FW', 'ST', 'CF']

# ลองหา position column
position_col = None
for col in df.columns:
    if 'pos' in str(col).lower():
        position_col = col
        break

print(f"📌 Position column found: {position_col}")

if position_col and position_col in df.columns:
    forwards_df = df[df[position_col].astype(str).str.contains('|'.join(forward_keywords), case=False, na=False)]
    print(f"✅ Found {len(forwards_df)} forwards")
else:
    print("⚠️ Could not find position column - using all players")
    forwards_df = df.copy()

## 🎯 Step 6: สร้าง TacticalFitAI Attributes

In [ ]:
# ดูว่ามี columns อะไรบ้าง
print("🔍 Searching for relevant columns...\n")

# แสดง columns ที่เกี่ยวข้อง
keywords = ['goal', 'shot', 'pass', 'press', 'xg', 'tackle', 'prog', 'foul', 'card']
for keyword in keywords:
    matching_cols = [col for col in forwards_df.columns if keyword in str(col).lower()]
    if matching_cols:
        print(f"📌 {keyword.upper()}: {matching_cols[:5]}")

In [ ]:
# สร้าง DataFrame สำหรับ TacticalFitAI
print("\n🎨 Creating TacticalFitAI dataset with IMPROVED FORMULAS...\n")
print("✨ Using formulas from fix.md for more realistic values\n")

tactical_df = pd.DataFrame()

# Player name
player_col = None
for col in forwards_df.columns:
    if 'player' in str(col).lower():
        player_col = col
        break

if player_col:
    tactical_df['Player'] = forwards_df[player_col]
else:
    print("⚠️ Could not find player column")
    tactical_df['Player'] = ''

# Position
tactical_df['Position'] = 'ST'

# ============ Finishing - 🔥 NEW FORMULA from fix.md ============
# Formula: Finishing = 70 + ((goals_per90 / xG_per90) * 20)
# สะท้อน "ยิงคมกว่าโอกาส"
goals_col = None
xg_col = None

for col in forwards_df.columns:
    col_str = str(col).lower()
    if ('gls' in col_str or 'goal' in col_str) and '90' in col_str and 'xg' not in col_str:
        goals_col = col
    if 'xg' in col_str and '90' in col_str and 'xa' not in col_str and 'assist' not in col_str:
        xg_col = col

print(f"📌 Finishing columns - Goals/90: {goals_col}, xG/90: {xg_col}")

if goals_col and xg_col and goals_col in forwards_df.columns and xg_col in forwards_df.columns:
    goals_per_90 = forwards_df[goals_col].fillna(0)
    xg_per_90 = forwards_df[xg_col].fillna(0.1)  # Avoid division by zero
    
    # NEW: ยิงได้มากกว่า xG = Finishing สูง (clinical finisher)
    finishing_ratio = (goals_per_90 / xg_per_90).clip(0.3, 2.0)  # Clip extreme values
    tactical_df['Finishing'] = 70 + (finishing_ratio * 20)
    
    # Add small noise
    np.random.seed(42)
    noise = np.random.normal(0, 2, len(tactical_df))
    tactical_df['Finishing'] = (tactical_df['Finishing'] + noise).clip(65, 95).round(0)
    
    tactical_df['xG'] = xg_per_90  # Store for later use
    print(f"   ✅ Range: {tactical_df['Finishing'].min():.0f} - {tactical_df['Finishing'].max():.0f}, Unique: {tactical_df['Finishing'].nunique()}")
else:
    tactical_df['Finishing'] = 75
    tactical_df['xG'] = 0.0
    print("   ⚠️ Using default value (75)")

# ============ Positioning - 🔥 NEW FORMULA from fix.md ============
# Formula: Positioning = 60 + (xG_per90 / max_xG_per90) * 40
# ปลอดภัยและเสถียรกว่า log
print(f"📌 Positioning - using xG normalization")

if 'xG' in tactical_df.columns and tactical_df['xG'].max() > 0:
    max_xg = tactical_df['xG'].max()
    tactical_df['Positioning'] = 60 + (tactical_df['xG'] / max_xg) * 40
    
    # Add noise
    np.random.seed(43)
    noise = np.random.normal(0, 2, len(tactical_df))
    tactical_df['Positioning'] = (tactical_df['Positioning'] + noise).clip(65, 95).round(0)
    print(f"   ✅ Range: {tactical_df['Positioning'].min():.0f} - {tactical_df['Positioning'].max():.0f}, Unique: {tactical_df['Positioning'].nunique()}")
else:
    tactical_df['Positioning'] = 75
    print("   ⚠️ Using default value (75)")

# ============ Speed - 🔥 NEW FORMULA from fix.md ============
# Formula: Speed = 65 + (0.6*z(prog_carries) + 0.4*z(dribbles)) * 10
# ใช้ z-score normalization
prog_carr_col = None
succ_dribble_col = None

for col in forwards_df.columns:
    col_str = str(col).lower()
    if 'prgc' in col_str.replace(' ', '') or ('prog' in col_str and 'carr' in col_str):
        prog_carr_col = col
    if ('succ' in col_str and 'drib' in col_str) or ('successful' in col_str and 'dribbles' in col_str):
        succ_dribble_col = col

print(f"📌 Speed columns - PrgCarr: {prog_carr_col}, SuccDrib: {succ_dribble_col}")

if prog_carr_col and prog_carr_col in forwards_df.columns:
    prog_carr = forwards_df[prog_carr_col].fillna(0)
    
    # Z-score normalization
    prog_carr_mean = prog_carr.mean()
    prog_carr_std = prog_carr.std() + 0.01  # Avoid division by zero
    z_prog_carr = (prog_carr - prog_carr_mean) / prog_carr_std
    
    if succ_dribble_col and succ_dribble_col in forwards_df.columns:
        succ_drib = forwards_df[succ_dribble_col].fillna(0)
        drib_mean = succ_drib.mean()
        drib_std = succ_drib.std() + 0.01
        z_drib = (succ_drib - drib_mean) / drib_std
        
        # Combined formula: 60% carries + 40% dribbles
        tactical_df['Speed'] = 65 + (0.6 * z_prog_carr + 0.4 * z_drib) * 10
    else:
        tactical_df['Speed'] = 65 + (z_prog_carr * 12)
    
    # Add noise
    np.random.seed(46)
    noise = np.random.normal(0, 2, len(tactical_df))
    tactical_df['Speed'] = (tactical_df['Speed'] + noise).clip(60, 95).round(0)
    print(f"   ✅ Range: {tactical_df['Speed'].min():.0f} - {tactical_df['Speed'].max():.0f}, Unique: {tactical_df['Speed'].nunique()}")
else:
    np.random.seed(42)
    tactical_df['Speed'] = np.random.normal(75, 8, len(tactical_df)).clip(60, 95).round(0)
    print("   ⚠️ Using estimated values (60-95)")

# ============ Strength - 🔥 NEW FORMULA from fix.md ============
# Formula: Strength = 60 + ((aerial_win_pct * aerials_per90) / max) * 40
# คูณด้วยปริมาณดวล (เพื่อให้คนที่ชนะน้อยแต่ดวลเยอะไม่ได้เปรียบ)
aerial_pct_col = None
aerial_won_col = None

for col in forwards_df.columns:
    col_str = str(col).lower()
    if 'aerial' in col_str and ('won%' in col_str or '%' in col_str):
        aerial_pct_col = col
    if 'aerial' in col_str and 'won' in col_str and '%' not in col_str and '90' in col_str:
        aerial_won_col = col

print(f"📌 Strength columns - Aerial%: {aerial_pct_col}, AerialWon/90: {aerial_won_col}")

if aerial_pct_col and aerial_pct_col in forwards_df.columns:
    aerial_pct = forwards_df[aerial_pct_col].fillna(50)
    
    if aerial_won_col and aerial_won_col in forwards_df.columns:
        # NEW: คูณด้วยจำนวนครั้งที่ดวล
        aerial_won_per90 = forwards_df[aerial_won_col].fillna(0)
        strength_metric = (aerial_pct / 100) * aerial_won_per90
        max_strength = strength_metric.max() + 0.01
        tactical_df['Strength'] = 60 + (strength_metric / max_strength) * 40
    else:
        # Fallback: ใช้แค่ %
        tactical_df['Strength'] = 55 + (aerial_pct - 30) * 0.7
    
    tactical_df['Strength'] = tactical_df['Strength'].clip(60, 95).round(0)
    print(f"   ✅ Range: {tactical_df['Strength'].min():.0f} - {tactical_df['Strength'].max():.0f}")
else:
    np.random.seed(43)
    tactical_df['Strength'] = np.random.normal(75, 8, len(tactical_df)).clip(60, 95).round(0)
    print("   ⚠️ Using random values (60-95)")

# ============ Passing - 🔥 NEW FORMULA from fix.md ============
# Formula: Passing = 0.4*Pass% + 0.3*ProgPass_norm + 0.3*KeyPass_norm
# รวม progressive passes + key passes
pass_pct_col = None
prog_pass_col = None
key_pass_col = None

for col in forwards_df.columns:
    col_str = str(col).lower()
    if 'pass' in col_str and 'cmp%' in col_str.replace(' ', ''):
        pass_pct_col = col
    if 'prgp' in col_str.replace(' ', '') or ('prog' in col_str and 'pass' in col_str and 'received' not in col_str):
        prog_pass_col = col
    if 'kp' in col_str and '90' in col_str:
        key_pass_col = col

print(f"📌 Passing columns - Pass%: {pass_pct_col}, PrgP: {prog_pass_col}, KP: {key_pass_col}")

if pass_pct_col and pass_pct_col in forwards_df.columns:
    pass_pct = forwards_df[pass_pct_col].fillna(70)
    
    # Start with pass completion %
    passing_score = 0.4 * pass_pct
    
    # Add progressive passes (normalized)
    if prog_pass_col and prog_pass_col in forwards_df.columns:
        prog_pass = forwards_df[prog_pass_col].fillna(0)
        max_prog = prog_pass.max() + 0.01
        prog_norm = (prog_pass / max_prog) * 100
        passing_score += 0.3 * prog_norm
    else:
        passing_score += 0.3 * 50  # Default
    
    # Add key passes (normalized)
    if key_pass_col and key_pass_col in forwards_df.columns:
        key_pass = forwards_df[key_pass_col].fillna(0)
        max_kp = key_pass.max() + 0.01
        kp_norm = (key_pass / max_kp) * 100
        passing_score += 0.3 * kp_norm
    else:
        passing_score += 0.3 * 50  # Default
    
    tactical_df['Passing'] = passing_score.clip(55, 95).round(0)
    print(f"   ✅ Range: {tactical_df['Passing'].min():.0f} - {tactical_df['Passing'].max():.0f}")
else:
    tactical_df['Passing'] = 70
    print("   ⚠️ Using default value (70)")

# ============ Vision - 🔥 NEW FORMULA from fix.md ============
# Formula: Vision = 60 + ((prog_passes_per90 + key_passes_per90*0.8) / max) * 40
print(f"📌 Vision - using normalized progressive + key passes")

if prog_pass_col and prog_pass_col in forwards_df.columns:
    prog_pass = forwards_df[prog_pass_col].fillna(0)
    
    if key_pass_col and key_pass_col in forwards_df.columns:
        key_pass = forwards_df[key_pass_col].fillna(0)
        vision_metric = prog_pass + (key_pass * 0.8)
    else:
        vision_metric = prog_pass
    
    max_vision = vision_metric.max() + 0.01
    tactical_df['Vision'] = 60 + (vision_metric / max_vision) * 40
    
    # Add noise
    np.random.seed(44)
    noise = np.random.normal(0, 2, len(tactical_df))
    tactical_df['Vision'] = (tactical_df['Vision'] + noise).clip(60, 95).round(0)
    print(f"   ✅ Range: {tactical_df['Vision'].min():.0f} - {tactical_df['Vision'].max():.0f}, Unique: {tactical_df['Vision'].nunique()}")
else:
    tactical_df['Vision'] = 70
    print("   ⚠️ Using default value (70)")

# ============ Aggression - 🔥 NEW FORMULA from fix.md ============
# Formula: Aggression = 70 + ((pressures_per90 + tackles_won_per90) / max) * 30
# ใช้ "Press Intensity" แทนฟาวล์
press_col = None
tackle_won_col = None

# Search for pressures
search_patterns = [
    lambda c: 'press' in c and '90' in c,
    lambda c: 'pressures' in c,
]

for pattern in search_patterns:
    for col in forwards_df.columns:
        col_str = str(col).lower()
        if pattern(col_str):
            press_col = col
            break
    if press_col:
        break

# Search for tackles won
for col in forwards_df.columns:
    col_str = str(col).lower()
    if 'tkl' in col_str and 'won' in col_str:
        tackle_won_col = col
        break
    if 'tackles' in col_str and 'won' in col_str:
        tackle_won_col = col
        break

print(f"📌 Aggression columns - Press: {press_col}, TklWon: {tackle_won_col}")

if press_col and press_col in forwards_df.columns:
    press_data = forwards_df[press_col].fillna(0)
    
    if tackle_won_col and tackle_won_col in forwards_df.columns:
        tackle_data = forwards_df[tackle_won_col].fillna(0)
        aggression_metric = press_data + tackle_data
    else:
        aggression_metric = press_data
    
    max_agg = aggression_metric.max() + 0.01
    tactical_df['Aggression'] = 70 + (aggression_metric / max_agg) * 30
    tactical_df['Aggression'] = tactical_df['Aggression'].clip(65, 95).round(0)
    print(f"   ✅ Range: {tactical_df['Aggression'].min():.0f} - {tactical_df['Aggression'].max():.0f}")
else:
    print("   ⚠️ Column not found - will calculate in post-processing")
    tactical_df['Aggression'] = 75

# ============ Composure - 🔥 NEW FORMULA from fix.md ============
# Formula: Composure = 75 + ((npxG90_goals - npxG90).clip(-0.3, 0.3)) * 40
# ใช้ per90 + cap ค่าเพื่อไม่ให้สุดโต่ง
print(f"📌 Composure - using capped overperformance per 90")

if goals_col and xg_col and goals_col in forwards_df.columns and xg_col in forwards_df.columns:
    goals_per90 = forwards_df[goals_col].fillna(0)
    xg_per90 = forwards_df[xg_col].fillna(0)
    
    # Overperformance with strict clipping
    overperf = (goals_per90 - xg_per90).clip(-0.3, 0.3)
    tactical_df['Composure'] = 75 + (overperf * 40)
    
    # Add small noise
    np.random.seed(47)
    noise = np.random.normal(0, 1.5, len(tactical_df))
    tactical_df['Composure'] = (tactical_df['Composure'] + noise).clip(65, 95).round(0)
    print(f"   ✅ Range: {tactical_df['Composure'].min():.0f} - {tactical_df['Composure'].max():.0f}")
else:
    tactical_df['Composure'] = 75
    print("   ⚠️ Using default value (75)")

# ============ OffTheBall - 🔥 NEW FORMULA from fix.md ============
# Formula: OffTheBall = 65 + (touches_in_box_per90 / max) * 35
# ใช้ touches in box (หรือ attacking 3rd touches)
touches_box_col = None
touches_att_col = None

for col in forwards_df.columns:
    col_str = str(col).lower()
    if 'touches' in col_str and ('box' in col_str or 'pen' in col_str) and '90' in col_str:
        touches_box_col = col
    if 'touches' in col_str and ('att' in col_str or 'final' in col_str) and '90' in col_str:
        touches_att_col = col

print(f"📌 OffTheBall columns - TouchesBox: {touches_box_col}, TouchesAtt: {touches_att_col}")

if touches_box_col and touches_box_col in forwards_df.columns:
    touches_box = forwards_df[touches_box_col].fillna(0)
    max_touches = touches_box.max() + 0.01
    tactical_df['OffTheBall'] = 65 + (touches_box / max_touches) * 35
    
    # Add noise
    np.random.seed(45)
    noise = np.random.normal(0, 2, len(tactical_df))
    tactical_df['OffTheBall'] = (tactical_df['OffTheBall'] + noise).clip(65, 95).round(0)
    print(f"   ✅ Range: {tactical_df['OffTheBall'].min():.0f} - {tactical_df['OffTheBall'].max():.0f}, Unique: {tactical_df['OffTheBall'].nunique()}")
elif touches_att_col and touches_att_col in forwards_df.columns:
    # Fallback: use attacking third touches
    touches_att = forwards_df[touches_att_col].fillna(0)
    max_touches = touches_att.max() + 0.01
    tactical_df['OffTheBall'] = 65 + (touches_att / max_touches) * 30
    tactical_df['OffTheBall'] = tactical_df['OffTheBall'].clip(65, 95).round(0)
    print(f"   ✅ Using attacking touches - Range: {tactical_df['OffTheBall'].min():.0f} - {tactical_df['OffTheBall'].max():.0f}")
else:
    # Final fallback: use progressive carries
    if prog_carr_col and prog_carr_col in forwards_df.columns:
        prog_carr = forwards_df[prog_carr_col].fillna(0)
        max_carr = prog_carr.max() + 0.01
        tactical_df['OffTheBall'] = 65 + (prog_carr / max_carr) * 30
        tactical_df['OffTheBall'] = tactical_df['OffTheBall'].clip(65, 95).round(0)
        print(f"   ⚠️ Using progressive carries - Range: {tactical_df['OffTheBall'].min():.0f} - {tactical_df['OffTheBall'].max():.0f}")
    else:
        tactical_df['OffTheBall'] = 75
        print("   ⚠️ Using default value (75)")

# ============ PressActions - 🔥 NEW FORMULA from fix.md ============
# Formula: PressActions = (pressures_per90 + tackles_won_per90 * 0.5)
# ใช้สถิติจริง แล้ว normalize 0-100 ภายหลัง
print(f"📌 PressActions - using real statistics")

if press_col and press_col in forwards_df.columns:
    press_data = forwards_df[press_col].fillna(0)
    
    if tackle_won_col and tackle_won_col in forwards_df.columns:
        tackle_data = forwards_df[tackle_won_col].fillna(0)
        tactical_df['PressActions'] = (press_data + tackle_data * 0.5).clip(0, 25).round(1)
    else:
        tactical_df['PressActions'] = press_data.clip(0, 25).round(1)
    
    print(f"   ✅ Range: {tactical_df['PressActions'].min():.1f} - {tactical_df['PressActions'].max():.1f}")
else:
    print("   ⚠️ Column not found - will calculate in post-processing")
    tactical_df['PressActions'] = 7.0

print("\n✅ TacticalFitAI dataset created with IMPROVED formulas!")
print(f"📊 Shape: {tactical_df.shape}")

## 🧹 Step 7: Clean และ Filter Data

In [ ]:
# Remove players with missing names
tactical_df = tactical_df[tactical_df['Player'].notna()]
tactical_df = tactical_df[tactical_df['Player'] != '']

# Remove duplicates
tactical_df = tactical_df.drop_duplicates(subset=['Player'], keep='first')

# ============ 🔥 UPDATED: More players + Lower xG threshold ============
# Filter เฉพาะนักเตะที่มี xG > 0.05 (เดิมคือ 0.1)
# ลดลงเพื่อให้ได้นักเตะดังๆ เช่น Mbappé, Haaland, Kane
tactical_df = tactical_df[tactical_df['xG'] > 0.05]  # ลดจาก 0.1 → 0.05

# Sort by xG
tactical_df = tactical_df.sort_values('xG', ascending=False)

# Keep top 100 (เดิมคือ 50)
tactical_df = tactical_df.head(100)  # เพิ่มจาก 50 → 100 คน

print(f"🎯 Filtered to top {len(tactical_df)} players (xG > 0.05)")

# ============ 🔧 POST-PROCESSING: Fix Default Values ============
print("\n🔧 Post-processing attributes with default values...")

# Check if PressActions has no variation (all same value)
if tactical_df['PressActions'].nunique() == 1:
    print("⚠️  PressActions has no variation - calculating from Speed & OffTheBall...")
    base_press = 7.5
    speed_norm = (tactical_df['Speed'] - tactical_df['Speed'].min()) / (tactical_df['Speed'].max() - tactical_df['Speed'].min() + 0.01)
    offball_norm = (tactical_df['OffTheBall'] - tactical_df['OffTheBall'].min()) / (tactical_df['OffTheBall'].max() - tactical_df['OffTheBall'].min() + 0.01)
    
    tactical_df['PressActions'] = base_press + (speed_norm * 3) + (offball_norm * 2)
    noise = np.random.normal(0, 0.5, len(tactical_df))
    tactical_df['PressActions'] = (tactical_df['PressActions'] + noise).clip(4.5, 15.0).round(1)
    print(f"   ✅ Fixed! Range: {tactical_df['PressActions'].min():.1f}-{tactical_df['PressActions'].max():.1f}, Unique: {tactical_df['PressActions'].nunique()}")

# Check if Aggression has no variation
if tactical_df['Aggression'].nunique() == 1:
    print("⚠️  Aggression has no variation - calculating from Strength & PressActions...")
    base_agg = 75
    strength_norm = (tactical_df['Strength'] - tactical_df['Strength'].min()) / (tactical_df['Strength'].max() - tactical_df['Strength'].min() + 0.01)
    press_norm = (tactical_df['PressActions'] - tactical_df['PressActions'].min()) / (tactical_df['PressActions'].max() - tactical_df['PressActions'].min() + 0.01)
    
    tactical_df['Aggression'] = base_agg + (strength_norm * 8) + (press_norm * 5)
    noise = np.random.normal(0, 2, len(tactical_df))
    tactical_df['Aggression'] = (tactical_df['Aggression'] + noise).clip(68, 92).round(0)
    print(f"   ✅ Fixed! Range: {tactical_df['Aggression'].min():.0f}-{tactical_df['Aggression'].max():.0f}, Unique: {tactical_df['Aggression'].nunique()}")

print(f"\n✅ Final dataset: {len(tactical_df)} players")
print(f"\n🎯 Top 5 players by xG:")
print(tactical_df[['Player', 'xG', 'Finishing', 'Positioning']].head())

## 💾 Step 8: Export เป็น CSV

In [ ]:
# Export
output_file = 'tacticalfitai_real_data.csv'
tactical_df.to_csv(output_file, index=False)

print(f"✅ Data exported to: {output_file}")
print(f"📥 Download this file and replace your data/players.csv")

# ============ 📊 COMPREHENSIVE SUMMARY ============
print(f"\n{'='*70}")
print("📊 FINAL DATASET SUMMARY")
print(f"{'='*70}")

print(f"\n🎯 Dataset Info:")
print(f"   Total Players: {len(tactical_df)}")
print(f"   Total Attributes: {len(tactical_df.columns) - 2}") # Exclude Player and Position

print(f"\n📈 Attribute Quality Check:")
print(f"{'Attribute':<15} {'Unique':>8} {'Range':>15} {'Average':>10} {'Status'}")
print(f"{'-'*70}")

for col in ['Finishing', 'Positioning', 'Speed', 'Strength', 'Passing', 'PressActions', 'Vision', 'Aggression', 'Composure', 'OffTheBall']:
    unique = tactical_df[col].nunique()
    min_val = tactical_df[col].min()
    max_val = tactical_df[col].max()
    mean_val = tactical_df[col].mean()
    status = '✅ Good' if unique > 10 else '⚠️  Fair' if unique > 5 else '⚠️  Low'
    print(f"{col:<15} {unique:>8} {min_val:>6.1f}-{max_val:<6.1f} {mean_val:>10.1f} {status}")

# Check for famous players
print(f"\n⚽ Famous Players Verification:")
famous = ['Erling Haaland', 'Kylian Mbappé', 'Harry Kane', 'Robert Lewandowski', 
          'Víctor Osimhen', 'Lautaro Martínez']
found = 0
for player in famous:
    if player in tactical_df['Player'].values:
        found += 1
        row = tactical_df[tactical_df['Player'] == player].iloc[0]
        print(f"   ✅ {player}: Fin={row['Finishing']:.0f}, xG={row['xG']:.2f}")

print(f"\n   Found {found}/{len(famous)} famous players")

# Overall quality score
total_unique = sum([tactical_df[col].nunique() for col in ['Finishing', 'Positioning', 'Speed', 'Strength', 'Passing', 'PressActions', 'Vision', 'Aggression', 'Composure', 'OffTheBall']])
quality_score = min(100, (total_unique / 150) * 100)  # 150 = ideal (15 unique per attr * 10 attrs)

print(f"\n🏆 Overall Data Quality Score: {quality_score:.0f}/100")
if quality_score >= 80:
    print("   ✅ EXCELLENT - Ready for production!")
elif quality_score >= 60:
    print("   ⚠️  GOOD - Minor improvements possible")
else:
    print("   ⚠️  FAIR - Consider adding more data sources")

print(f"\n{'='*70}")

## 🎨 Step 9: Preview Data

In [ ]:
# Display full dataset
print("\n📋 Full Dataset Preview:")
tactical_df

## 🚀 Step 10 (Optional): เพิ่มข้อมูลจาก SoFIFA สำหรับ Speed & Strength

In [ ]:
# ถ้าต้องการ Speed และ Strength ที่แม่นยำกว่า
# ต้องใช้ SoFIFA (แต่ต้อง manual map ชื่อนักเตะ)

print("⚠️ SoFIFA integration: Manual process required")
print("Steps:")
print("1. ไปที่ https://sofifa.com")
print("2. Export player ratings")
print("3. Match player names manually")
print("4. Update Speed & Strength columns")

---
## ✅ เสร็จสิ้น!

**Next Steps:**
1. Download `tacticalfitai_real_data.csv`
2. Replace `data/players.csv` ในโปรเจกต์
3. Run `streamlit run app_advanced.py`
4. เพลิดเพลินกับข้อมูลจริง! 🎉